# Post Processing

This notebook run some post processing on result point cloud. 
1. Convert point cloud to DEM with `point2dem` command from ASP.
2. Coregister with reference DEMs.
3. Compute difference between coregistered DEMs and references DEMs.
4. Compute some statistics of this difference.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import history

/workdir/cryodyn/dehecqa/bin/miniforge3/envs/history/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# For testing only

Temporay commands for testing:
- create a symlink to the files needed for this notebook
- create downsampled version of the reference DEMs

In [3]:
# !ln -fs /mnt/summer/USERS/DEHECQA/history/output/extracted/**/*dense_pointcloud.laz /mnt/summer/USERS/DEHECQA/history/output/test_data/
!ls -1 /mnt/summer/USERS/DEHECQA/history/output/test_data/

ASPY_CG_MC_PP_CN_GN_PN_MN_dense_pointcloud.laz
casa_grande_reference_dem_zoom_10m.tif
casa_grande_reference_dem_zoom_mask_10m.tif
dems
ESM_CG_AI_PP_CN_GY_PN_MN_dense_pointcloud.laz
ESM_CG_AI_RA_CN_GY_PN_MN_dense_pointcloud.laz
ESM_CG_AI_RA_CY_GY_PN_MN_dense_pointcloud.laz
ESM_IL_AI_PP_CN_GY_PN_MN_dense_pointcloud.laz
ESM_IL_AI_RA_CN_GY_PN_MN_dense_pointcloud.laz
ESM_IL_AI_RA_CY_GY_PN_MN_dense_pointcloud.laz
fdahle_CG_AI_PP_CN_GY_PN_MN_dense_pointcloud.laz
fdahle_CG_AI_PP_CY_GN_PN_MN_dense_pointcloud.laz
fdahle_CG_AI_PP_CY_GY_PN_MN_dense_pointcloud.laz
fdahle_IL_AI_PP_CY_GN_PN_MN_dense_pointcloud.laz
fdahle_IL_AI_PP_CY_GY_PN_MN_dense_pointcloud.laz
HIATUS_CG_AI_PP_CY_PN_MN_dense_pointcloud.laz
HIATUS_CG_AI_PP_CY_PN_MY_dense_pointcloud.laz
HSfM_CG_AI_PP_CY_GY_PY_MN_dense_pointcloud.laz
HSfM_IL_AI_PP_CY_GY_PN_MN_dense_pointcloud.laz
JB_CG_AI_PP_CY_GN_PN_MN_dense_pointcloud.las
JB_CG_AI_PP_CY_GN_PN_MN_sparse_pointcloud.las
TUW_CG_AI_PP_CN_GN_PN_MN_dense_pointcloud.laz
TUW_IL_AI_PP_CN_GN_PN

In [4]:
#! gdal_translate /mnt/summer/USERS/DEHECQA/history/data_final/casa_grande/aux_data/reference_dem_zoom.tif test_data/casa_grande_reference_dem_zoom_10m.tif -tr 10 10 -r cubic
#!gdal_translate /mnt/summer/USERS/DEHECQA/history/data_final/casa_grande/aux_data/reference_dem_zoom_mask.tif test_data/casa_grande_reference_dem_zoom_mask_10m.tif -tr 10 10 -r cubic

# Settings

In [9]:
# contain the .las or .laz files
POINTCLOUD_DIRECTORY = "/mnt/summer/USERS/DEHECQA/history/output/test_data"

# Directory where put DEMs after point2dem command
RAW_DEM_DIRECTORY = "/mnt/summer/USERS/DEHECQA/history/output/test_data/dems"

# Directory of coregistered DEMs
COREGISTERED_DEM_DIRECTORY = "/mnt/summer/USERS/DEHECQA/history/output/test_data/coregistered_dems"

# Directory of diference beetween ref DEMs and coregistered DEMs
DDEMS_DIRECTORY = "/mnt/summer/USERS/DEHECQA/history/output/test_data/ddems"

# dictionnary of all references DEMs (some value can be set to None depend on site and dataset)
REF_DEMS = {
    # "casagrande_ref_dem_zoom" : "/mnt/summer/USERS/DEHECQA/history/data_final/casa_grande/aux_data/reference_dem_zoom.tif",
    "casagrande_ref_dem_zoom" : "/mnt/summer/USERS/DEHECQA/history/output/test_data/casa_grande_reference_dem_zoom_10m.tif",
    "casagrande_ref_dem_large" : None,
    "iceland_ref_dem_zoom" : None,
    "iceland_ref_dem_large" : None
}

# dictionnary of all references DEMs masks (some value can be set to None depend on site and dataset)
REF_DEM_MASKS = {
    # "casagrande_ref_dem_zoom_mask" : "/mnt/summer/USERS/DEHECQA/history/data_final/casa_grande/aux_data/reference_dem_zoom_mask.tif",
    "casagrande_ref_dem_zoom_mask" : "/mnt/summer/USERS/DEHECQA/history/output/test_data/casa_grande_reference_dem_zoom_mask_10m.tif",
    "casagrande_ref_dem_large_mask" : None,
    "iceland_ref_dem_zoom_mask" : None,
    "iceland_ref_dem_large_mask" : None
}

OVERWRITE = False

# set this to True to avoid process
DRY_RUN = False

MAX_WORKERS = 4

## Converting cloud point to DEM

Batch process point cloud files in a directory to generate DEMs aligned with reference DEMs.

This function iterates over all point cloud files (*.las or *.laz) in `input_directory`,
selects the appropriate reference DEM based on site and dataset extracted from filenames,
and calls `point2dem` to create coregistered DEMs saved in `output_directory`.

The `max_workers` arg set a process for each point2dem command so for one file it's useless

Different attempts to add ASP to path, without success...

In [6]:
# !module load stereo3.5.0
# !export PATH=$PATH:/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/
# !point2dem -h
# !echo $PATH

In [7]:
# import subprocess
# subprocess.run("export PATH=$PATH:/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/", shell=True, check=True)
# subprocess.run("echo $PATH", shell=True, check=True)

In [14]:
history.postprocessing.iter_point2dem(POINTCLOUD_DIRECTORY, RAW_DEM_DIRECTORY, **REF_DEMS, overwrite=OVERWRITE, dry_run=DRY_RUN, max_workers=MAX_WORKERS, asp_path="/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/")

Skip JB_CG_AI_PP_CY_GN_PN_MN_dense_pointcloud.las : /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/JB_CG_AI_PP_CY_GN_PN_MN_dense-DEM.tif already exist.
Skip JB_CG_AI_PP_CY_GN_PN_MN_sparse_pointcloud.las : /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/JB_CG_AI_PP_CY_GN_PN_MN_sparse-DEM.tif already exist.
Skip ASPY_CG_MC_PP_CN_GN_PN_MN_dense_pointcloud.laz : No reference DEM provided (site: CG, dataset: MC).
Skip ESM_CG_AI_PP_CN_GY_PN_MN_dense_pointcloud.laz : /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/ESM_CG_AI_PP_CN_GY_PN_MN_dense-DEM.tif already exist.
Skip ESM_CG_AI_RA_CN_GY_PN_MN_dense_pointcloud.laz : /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/ESM_CG_AI_RA_CN_GY_PN_MN_dense-DEM.tif already exist.
Skip ESM_CG_AI_RA_CY_GY_PN_MN_dense_pointcloud.laz : /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/ESM_CG_AI_RA_CY_GY_PN_MN_dense-DEM.tif already exist.
Skip ESM_IL_AI_PP_CN_GY_PN_MN_dense_pointcloud.laz : No reference DEM provided (si

Process ForkProcess-8:
Traceback (most recent call last):
  File "/workdir/cryodyn/dehecqa/bin/miniforge3/envs/history/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/workdir/cryodyn/dehecqa/bin/miniforge3/envs/history/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/workdir/cryodyn/dehecqa/bin/miniforge3/envs/history/lib/python3.11/concurrent/futures/process.py", line 249, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workdir/cryodyn/dehecqa/bin/miniforge3/envs/history/lib/python3.11/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
          ^^^^^^^^^^^^^^^^^^
  File "/workdir/cryodyn/dehecqa/bin/miniforge3/envs/history/lib/python3.11/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workdir/cry

KeyboardInterrupt: 

## Coregister DEMs

Coregister multiple DEMs in a directory to appropriate reference DEMs and return summary statistics.

This function iterates over all DEM files in `input_directory` ending with '-DEM.tif', 
selects the appropriate reference DEM and mask based on the site and dataset information,
and applies coregistration using the `coregister_dem` function. 

Coregistered DEMs are saved to `output_directory`. If `overwrite` is False, existing output files are skipped.
If `dry_run` is True, no coregistration is performed, only file names and planned operations are printed.

In [10]:
result_df = history.postprocessing.iter_coregister_dems(
    RAW_DEM_DIRECTORY, COREGISTERED_DEM_DIRECTORY, 
    **REF_DEMS, **REF_DEM_MASKS, overwrite=OVERWRITE, dry_run=DRY_RUN, output_ddem_after_directory=DDEMS_DIRECTORY
)
result_df

coregister_dem(/mnt/summer/USERS/DEHECQA/history/output/test_data/dems/HSfM_CG_AI_PP_CY_GY_PY_MN_dense-DEM.tif, /mnt/summer/USERS/DEHECQA/history/output/test_data/casa_grande_reference_dem_zoom_10m.tif, /mnt/summer/USERS/DEHECQA/history/output/test_data/casa_grande_reference_dem_zoom_mask_10m.tif, /mnt/summer/USERS/DEHECQA/history/output/test_data/coregistered_dems/HSfM_CG_AI_PP_CY_GY_PY_MN_dense-DEM.tif)
coregister_dem(/mnt/summer/USERS/DEHECQA/history/output/test_data/dems/JB_CG_AI_PP_CY_GN_PN_MN_sparse-DEM.tif, /mnt/summer/USERS/DEHECQA/history/output/test_data/casa_grande_reference_dem_zoom_10m.tif, /mnt/summer/USERS/DEHECQA/history/output/test_data/casa_grande_reference_dem_zoom_mask_10m.tif, /mnt/summer/USERS/DEHECQA/history/output/test_data/coregistered_dems/JB_CG_AI_PP_CY_GN_PN_MN_sparse-DEM.tif)


/workdir/cryodyn/dehecqa/bin/miniforge3/envs/history/lib/python3.11/site-packages/xdem/coreg/affine.py:507: RuntimeWarning: All-NaN slice encountered
  vshift = np.nanmedian(dh_step)


ValueError: The subsample contains no more valid values. This can happen is the horizontal shift to correct is very large, or if the algorithm diverged. To ensure all possible points can be used at any iteration step, use subsample=1.